# LoRa Fine-tuning Interactive Notebook

This notebook provides an interactive way to explore LoRa (Low-Rank Adaptation) fine-tuning.

## Overview
- **Dataset**: IMDb movie reviews
- **Task**: Sentiment classification
- **Model**: DistilBERT
- **Technique**: LoRa parameter-efficient fine-tuning

## 1. Setup and Imports

In [ ]:
# Install required packages for LoRa fine-tuning
# Run this cell once, then restart the kernel before proceeding

%pip install torch>=1.12.0 transformers>=4.21.0 datasets>=2.5.0 peft>=0.4.0
%pip install scikit-learn>=1.1.0 matplotlib>=3.5.0 numpy>=1.21.0 tqdm>=4.64.0
%pip install accelerate>=0.20.0 evaluate>=0.4.0 PyYAML>=6.0 ipywidgets>=8.0.0

print("🎉 Installation complete!")
print("🔄 IMPORTANT: Please restart the kernel (Kernel > Restart) before running the next cells.")
print("? After restarting, you can skip this cell and run the import cell below.")

Installing LoRa fine-tuning packages...
✅ Successfully installed torch>=1.12.0
✅ Successfully installed torch>=1.12.0
✅ Successfully installed transformers>=4.21.0
✅ Successfully installed transformers>=4.21.0
✅ Successfully installed datasets>=2.5.0
✅ Successfully installed datasets>=2.5.0
✅ Successfully installed peft>=0.4.0
✅ Successfully installed peft>=0.4.0
✅ Successfully installed scikit-learn>=1.1.0
✅ Successfully installed scikit-learn>=1.1.0
✅ Successfully installed matplotlib>=3.5.0
✅ Successfully installed matplotlib>=3.5.0
✅ Successfully installed numpy>=1.21.0
✅ Successfully installed numpy>=1.21.0
✅ Successfully installed tqdm>=4.64.0
✅ Successfully installed tqdm>=4.64.0
✅ Successfully installed accelerate>=0.20.0
✅ Successfully installed accelerate>=0.20.0
✅ Successfully installed evaluate>=0.4.0
✅ Successfully installed evaluate>=0.4.0
✅ Successfully installed PyYAML>=6.0
✅ Successfully installed PyYAML>=6.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 4

In [7]:
# Install required packages (run only once)
# !pip install torch transformers datasets peft scikit-learn matplotlib numpy tqdm accelerate

import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer
)
from datasets import load_dataset
from sklearn.metrics import accuracy_score, classification_report
from peft import get_peft_model, LoraConfig, TaskType
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Error importing huggingface_hub.hf_api: No module named 'yaml'


ModuleNotFoundError: No module named 'yaml'

## 2. Load Dataset

In [ ]:
# Load IMDb dataset
print("Loading IMDb dataset...")
dataset = load_dataset("imdb")

# Use smaller subsets for demo (increase for better results)
train_size = 1000
test_size = 200

train_data = dataset['train'].shuffle(seed=42).select(range(train_size))
test_data = dataset['test'].shuffle(seed=42).select(range(test_size))

print(f"Train samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")

# Show example
print("\nExample review:")
print(f"Text: {train_data[0]['text'][:200]}...")
print(f"Label: {train_data[0]['label']} ({'Positive' if train_data[0]['label'] == 1 else 'Negative'})")

## 3. Load Pre-trained Model

In [ ]:
model_name = "distilbert-base-uncased"
print(f"Loading {model_name}...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1}
)
model.to(device)

print(f"Model loaded with {model.num_parameters():,} parameters")

## 4. Evaluate Model Before Fine-tuning

In [ ]:
def quick_evaluate(model, tokenizer, texts, labels, max_samples=100):
    """Quick evaluation function"""
    model.eval()
    predictions = []
    
    # Limit samples for quick evaluation
    eval_texts = texts[:max_samples]
    eval_labels = labels[:max_samples]
    
    with torch.no_grad():
        for text in eval_texts:
            inputs = tokenizer(text, return_tensors="pt", truncation=True, 
                             padding=True, max_length=256).to(device)
            outputs = model(**inputs)
            prediction = torch.argmax(outputs.logits, dim=-1).item()
            predictions.append(prediction)
    
    accuracy = accuracy_score(eval_labels, predictions)
    return accuracy, predictions

print("Evaluating model BEFORE fine-tuning...")
pre_accuracy, pre_predictions = quick_evaluate(model, tokenizer, test_data['text'], test_data['label'])

print(f"\nAccuracy before fine-tuning: {pre_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(test_data['label'][:100], pre_predictions, 
                          target_names=['Negative', 'Positive']))

## 5. Setup LoRa Configuration

In [ ]:
# Configure LoRa
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Sequence classification
    inference_mode=False,
    r=16,                        # Rank - number of low-rank matrices
    lora_alpha=32,              # Scaling parameter
    lora_dropout=0.1,           # Dropout for regularization
    target_modules=["q_lin", "v_lin"],  # Target attention modules
)

# Apply LoRa to the model
model = get_peft_model(model, lora_config)

# Show trainable parameters
model.print_trainable_parameters()

## 6. Prepare Data for Training

In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=256)

print("Tokenizing datasets...")
train_dataset = train_data.map(tokenize_function, batched=True)
train_dataset = train_dataset.rename_column("label", "labels")

# Set format for PyTorch
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

print("Data preparation complete!")

## 7. Fine-tune with LoRa

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./lora_model",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=50,
    save_strategy="no",  # Don't save for demo
    remove_unused_columns=False,
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

print("Starting LoRa fine-tuning...")
trainer.train()
print("Fine-tuning completed!")

## 8. Evaluate Model After Fine-tuning

In [ ]:
print("Evaluating model AFTER fine-tuning...")
post_accuracy, post_predictions = quick_evaluate(model, tokenizer, test_data['text'], test_data['label'])

print(f"\nAccuracy after fine-tuning: {post_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(test_data['label'][:100], post_predictions, 
                          target_names=['Negative', 'Positive']))

## 9. Compare Results and Visualize

In [ ]:
# Results summary
improvement = post_accuracy - pre_accuracy
improvement_percent = improvement * 100

print("=" * 50)
print("RESULTS SUMMARY")
print("=" * 50)
print(f"Accuracy before fine-tuning: {pre_accuracy:.4f}")
print(f"Accuracy after fine-tuning:  {post_accuracy:.4f}")
print(f"Improvement: +{improvement:.4f} ({improvement_percent:.2f}%)")
print("=" * 50)

# Visualization
plt.figure(figsize=(10, 6))
accuracies = [pre_accuracy, post_accuracy]
labels = ['Before LoRa\nFine-tuning', 'After LoRa\nFine-tuning']
colors = ['#ff6b6b', '#4ecdc4']

bars = plt.bar(labels, accuracies, color=colors, alpha=0.8, edgecolor='black', linewidth=2)

# Add value labels on bars
for bar, accuracy in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{accuracy:.4f}', ha='center', va='bottom', fontweight='bold', fontsize=12)

plt.ylabel('Accuracy', fontsize=12, fontweight='bold')
plt.title('Model Performance: Before vs After LoRa Fine-tuning', fontsize=14, fontweight='bold')
plt.ylim(0, 1)
plt.grid(axis='y', alpha=0.3)

# Add improvement text
plt.text(0.5, 0.8, f'Improvement: +{improvement:.4f} ({improvement_percent:.2f}%)', 
         ha='center', transform=plt.gca().transAxes, fontsize=12, 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.7))

plt.tight_layout()
plt.show()

## 10. Test on Custom Examples

In [ ]:
def predict_sentiment(text, model, tokenizer):
    """Predict sentiment for a single text"""
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, 
                      padding=True, max_length=256).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=-1).item()
        confidence = torch.softmax(outputs.logits, dim=-1).max().item()
    
    sentiment = "Positive" if prediction == 1 else "Negative"
    return sentiment, confidence

# Test examples
test_examples = [
    "This movie was absolutely fantastic! I loved every minute of it.",
    "Worst movie I've ever seen. Complete waste of time.",
    "The movie was okay, nothing special but not terrible either.",
    "Amazing cinematography and brilliant acting. Highly recommend!",
    "Boring plot and poor character development. Very disappointed."
]

print("Testing on custom examples:")
print("=" * 60)

for i, text in enumerate(test_examples, 1):
    sentiment, confidence = predict_sentiment(text, model, tokenizer)
    print(f"{i}. Text: {text}")
    print(f"   Prediction: {sentiment} (Confidence: {confidence:.3f})")
    print()

## 11. Key Insights

### What we learned:

1. **Parameter Efficiency**: LoRa only trains <1% of the original model parameters
2. **Effectiveness**: Significant improvement in task performance
3. **Speed**: Much faster training compared to full fine-tuning
4. **Memory**: Lower GPU memory requirements

### LoRa Parameters Impact:
- **`r` (rank)**: Higher values = more parameters, potentially better performance
- **`lora_alpha`**: Controls the scaling of LoRa adaptation
- **`target_modules`**: Which layers to adapt (attention modules work well)

### Next Steps:
- Try different datasets and tasks
- Experiment with different LoRa configurations
- Compare with full fine-tuning
- Use larger models (e.g., Llama, GPT-style models)